<a href="https://colab.research.google.com/github/Manjushree329/BharathIntern_task2/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install pandas scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/Titanic_Data/')

In [ ]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
import pandas as pd

# Load the dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Display the first few rows of the training data
train_df.head()
# Combine train and test datasets for feature engineering
combined_df = pd.concat([train_df, test_df], sort=False)

# Extract titles from names
combined_df['Title'] = combined_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
combined_df['Title'] = combined_df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',
                                                     'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
combined_df['Title'] = combined_df['Title'].replace('Mlle', 'Miss')
combined_df['Title'] = combined_df['Title'].replace('Ms', 'Miss')
combined_df['Title'] = combined_df['Title'].replace('Mme', 'Mrs')

# Fill missing ages based on median age of titles
combined_df['Age'] = combined_df.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))

# Fill missing embarked with the most common port
combined_df['Embarked'] = combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0])

# Fill missing fare with median fare
combined_df['Fare'] = combined_df['Fare'].fillna(combined_df['Fare'].median())

# Create a new feature for family size
combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1

# Convert categorical features to numerical
combined_df = pd.get_dummies(combined_df, columns=['Sex', 'Embarked', 'Title'], drop_first=True)

# Drop irrelevant columns
combined_df.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1, inplace=True)

# Separate the combined dataset back into train and test sets
train_df = combined_df[:len(train_df)]
test_df = combined_df[len(train_df):]

# Restore the 'Survived' column to the training set
train_df['Survived'] = train_df['Survived'].astype(int)
from sklearn.model_selection import train_test_split

# Split the training data into features and target variable
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Predict on the validation set
y_pred = model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
conf_matrix = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')
# Predict on the test set
test_predictions = model.predict(test_df.drop('Survived', axis=1))

# Create a submission file
submission = pd.DataFrame({
    "PassengerId": pd.read_csv('test.csv')['PassengerId'],
    "Survived": test_predictions
})

# Save the submission to a CSV file
submission.to_csv('titanic_submission.csv', index=False)

<ipython-input-21-04898d9e727f>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Survived'] = train_df['Survived'].astype(int)


Accuracy: 0.8379888268156425
Confusion Matrix:
[[91 14]
 [15 59]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.87      0.86       105
           1       0.81      0.80      0.80        74

    accuracy                           0.84       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179

